# Predicting sentiment from product reviews

# Fire up GraphLab Create

In [1]:
import graphlab

# Read some product review data

Loading reviews for a set of baby products. 

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create is assigned to ykushla@gmail.com and will expire on May 25, 2017. For commercial licensing options, visit https://dato.com/buy/.


[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: C:\Users\ykushla\AppData\Local\Temp\graphlab_server_1466322549.log.0


# Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [3]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


# Build the word count vector for each review

In [4]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [5]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'blue-12': 1,'stripping': 1, 'about': ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'moist': 1, 'came': 1,'holder.': 1, 'i': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'this': 1, 'anyone': 1,'soft': 1, 'than': 1, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'this': 3, 'purchase.':1, 'much': 1, 'little': ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'this': 1, 'pacifier,':1, 'have': 1, 'for': 2, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'how': 1,'came': 1, 'binky': 3, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'be': 1, 'so': 1,'alot': 1, 'for': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'diaper': 1, 'life': 1,'half': 1, 'and': 2, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'this': 2, 'diaper': 1,'new': 1, 'i': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'this': 3, 'highly': 1,'have': 1, 'for': 1, ..."


In [6]:
graphlab.canvas.set_target('ipynb')

In [7]:
products['name'].show()

# Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [8]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [9]:
len(giraffe_reviews)

785

In [10]:
giraffe_reviews['rating'].show(view='Categorical')

# Build a sentiment classifier

In [11]:
products['rating'].show(view='Categorical')

## Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [12]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [13]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [14]:
products.head()

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'moist': 1, 'came': 1,'holder.': 1, 'i': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'this': 1, 'anyone': 1,'soft': 1, 'than': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'this': 3, 'purchase.':1, 'much': 1, 'little': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'this': 1, 'pacifier,':1, 'have': 1, 'for': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'how': 1,'came': 1, 'binky': 3, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'be': 1, 'so': 1,'alot': 1, 'for': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'diaper': 1, 'life': 1,'half': 1, 'and': 2, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'this': 2, 'diaper': 1,'new': 1, 'i': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'this': 3, 'highly': 1,'have': 1, 'for': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'rush': 1, 'quickly': 1,'well': 1, 'there': 1, ...",1


## Let's train the sentiment classifier

In [15]:
train_data,test_data = products.random_split(.8, seed=0)

In [17]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 4.775698     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 6.940453     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Evaluate the sentiment model

In [18]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0    |        1.0         |        1.0         | 27976 | 5328 |
 |   1e-05   | 0.9093468468468469 | 0.9988561624249357 | 27976 | 5328 |
 |   2e-05   | 0.896021021021021  | 0.9987489276522734 | 27976 | 5328 |
 |   3e-05   | 0.886448948948949  | 0.9984629682585073 | 27976 | 5328 |
 |   4e-05   | 0.8796921921921922 | 0.9982842436374035 | 27976 | 5328 |
 |   5e-05   | 0.8751876876876877 | 0.998212753788962  | 27976 | 5328 |
 |   6e-05   | 0.8721846846846847 | 0.9981770088647413 | 27976 | 5328 |
 |   7e-05   | 0.8686186186186187 | 0.9980340291678582 | 27976 | 5328 |
 |   8e-05   | 0.8646771771771772 | 0.9979982842436375 | 27976 | 5328 |
 |   9e-05   | 0.8607357357357

In [19]:
sentiment_model.show(view='Evaluation')

# Applying the learned model to understand sentiment for Giraffe

In [20]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [21]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'great': 1, 'helped': 1,'ears!': 1, 'purchase!': ...",0.9995130235210922
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'this': 1, 'diaper': 1,'great': 1, 'getting' ...",0.9993206783057708
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'this': 1, 'box.': 1,'repeated': 1, 'so': 1, ...",0.01355881168697714
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'this': 1, 'something':1, 'so': 1, 'have': 1, ...",0.995769474147639
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'this': 1, 'he': 2,'are': 1, 'everything': ...",0.6623744156733844
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'pack': 1, 'so': 1,'have': 1, 'sophie': 3, ...",0.9999971481859539
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'chewing,': 1, 'forth':1, 'sophie': 1, 'sque ...",0.9891909895355775
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'the': 1, 'sweet.': 1,'be': 1, 'so': 1, ...",0.9995635184126087
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'cram': 1, 'this': 1,'toxic.': 1, 'new': 1, ...",0.9701605427250727
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'this': 2, 'how': 2,'have': 2, 'teeth': 1, ...",0.1953676445880198


## Sort the reviews based on the predicted sentiment and explore

In [22]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [23]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'how': 2, 'much': 1,'nose': 1, 'few': 1, ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'loves': 2, 'quality':1, 'pile': 1, 'pleasa ...",0.9999999997027544
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'toy': 3, 'hooves': 1,'earned': 1, 'keepsake': ...",0.9999999993924809
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'quality': 1, 'sure': 1,'imaginable,': 1, 'ne ...",0.9999999991897695
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'found,': 1, 'much': 1,'stretching,': 1, 'can': ...",0.9999999986567458
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'new': 1, 'few': 1,'downside': 1, 'still': ...",0.9999999971078298
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'this': 2, 'know': 1,'much': 1, 'and': 5, ...",0.9999999955894878
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'car': 1, 'this': 3,'lol...': 1, 'sure': 1, ...",0.9999999955729006
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'toy!': 2, 'and': 4,'sophie': 2, 'for': 1, ...",0.99999998952692
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'nose': 1, 'so': 1, '2':1, 'squeezes': 1, ...",0.9999999850689071


## Most positive reviews for the giraffe

In [24]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [25]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

## Show most negative reviews for giraffe

In [26]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [27]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

# C13 - Assignment

In [28]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

## Task #1

In [42]:
sum_dict = {}

for word in selected_words:
    
    def f_count(word_dict):
        if word in word_dict:
            return word_dict[word]
        else:
            return 0
        
    products[word] = products['word_count'].apply(f_count)
    sum_dict[word] = products[word].sum()


In [44]:
products.head()

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'moist': 1, 'came': 1,'holder.': 1, 'i': 1, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'this': 1, 'anyone': 1,'soft': 1, 'than': 1, ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'this': 3, 'purchase.':1, 'much': 1, 'little': ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'this': 1, 'pacifier,':1, 'have': 1, 'for': 2, ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'how': 1,'came': 1, 'binky': 3, ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'be': 1, 'so': 1,'alot': 1, 'for': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'diaper': 1, 'life': 1,'half': 1, 'and': 2, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'this': 2, 'diaper': 1,'new': 1, 'i': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'this': 3, 'highly': 1,'have': 1, 'for': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'rush': 1, 'quickly': 1,'well': 1, 'there': 1, ...",1,0


In [45]:
sum_dict

{'amazing': 1305,
 'awesome': 2002,
 'awful': 345,
 'bad': 3197,
 'fantastic': 873,
 'great': 42420,
 'hate': 1057,
 'horrible': 659,
 'love': 40277,
 'terrible': 673,
 'wow': 131}

## Task #2

In [74]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.683464     | 0.844269          | 0.843142            |

| 5         | 6        | 0.834863     | 0.844269          | 0.843142            |

| 6         | 7        | 0.981253     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [75]:
selected_words_model['coefficients'].sort("value").print_rows(20)

+-------------+-------+-------+-----------------------+----------------------+
|     name    | index | class |         value         |        stderr        |
+-------------+-------+-------+-----------------------+----------------------+
|   terrible  |  None |   1   |   -2.090499984872608  |  0.0967241912228587  |
|   horrible  |  None |   1   |  -1.9965180055878038  | 0.09735841690278443  |
|    awful    |  None |   1   |  -1.7646995563132182  | 0.13467980336480187  |
|     hate    |  None |   1   |  -1.4091640627563853  |  0.0771983993505609  |
|     bad     |  None |   1   |  -0.9858273699287604  | 0.043360300914240736 |
|     wow     |  None |   1   | -0.054145012333251856 | 0.27561644941620517  |
|    great    |  None |   1   |   0.8839378948979686  | 0.021737952792139033 |
|  fantastic  |  None |   1   |   0.8913030903043073  | 0.15453234359087975  |
|   amazing   |  None |   1   |   0.8928024225084732  | 0.12798950323089522  |
|   awesome   |  None |   1   |   1.0580088887848116

## Task #3

In [76]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.916256305548883,
 'auc': 0.9446492867438502,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  1461 |
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9500349343413533,
 'log_loss': 0.26106698432421915,
 'precision': 0.9523039902309378,
 'recall': 0.9477766657134686,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.

In [77]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431419649291376,
 'auc': 0.6648096413721418,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      1       |        0        |  130  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.914242563530107,
 'log_loss': 0.405474711036565,
 'precision': 0.8453551912568306,
 'recall': 0.9953531598513011,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 1

In [78]:
products["sentiment"].mean()

0.841123344847434

## Task #4

In [55]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [56]:
len(diaper_champ_reviews)

298

In [62]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [63]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [64]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'changing': 1, 'can': 1,'chanp': 1, 'well': 1, ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'put': 1, 'how': 1,'much': 1, 'top': 1, ...",1,0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'sure': 1, 'so': 1,'changing': 1, 'bad': 1, ...",1,0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{'entry.': 1, 'much': 1,'sure': 1, 'extremely': ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'who': 1, 'much': 1,'new': 1, 'praises': 1, ...",1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'this': 1, 'can': 1,'runs': 1, 'bags.': 1, ...",1,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'plain': 1, 'this': 3,'ours.': 1, 'kitchen' ...",1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'how': 1, 'sure': 1,'so': 2, '2': 1, 'sti ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'using': 1, 'about': 2,'locks': 1, 'so': 2, ...",1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'much': 1, 'new': 1,'so': 5, ""day--it's"": 1, ...",1,0


In [65]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.796940851290671]

In [66]:
diaper_champ_reviews[0:1]

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float

Rows: 1

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'changing': 1, 'can': 1, ... |     1     |    0    |   0   |     0     |    0    |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+-----+------+---------------------+
| love | horrible | bad | terrible | awful | wow | hate | predicted_sentiment |
+------+----------+-----+----------+-------+-----+------+---------------------+
|  0   |    0     |  0  |    0     |   0   |  0  |  0   |  0.9999999372669541 |
+------+----------+-----+----------+-------+-----+------+---------------------+
[1 rows x 17 columns]

In [67]:
diaper_champ_reviews[0:1]["review"]

dtype: str
Rows: 1
['Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.']

In [73]:
diaper_champ_reviews[0:1]["word_count"][0]

{'"what': 1,
 '(which': 1,
 '3': 1,
 'a': 6,
 'absolutly': 2,
 'added': 1,
 'all': 1,
 'and': 6,
 'any': 1,
 'are': 1,
 'around': 1,
 'at': 1,
 'baby': 3,
 'bag': 1,
 'bag,': 1,
 'bags': 1,
 'bassinet': 1,
 'because': 1,
 'best': 2,
 'bjorn,': 1,
 'bulk': 1,
 'can': 1,
 'champ': 1,
 'champ,': 2,
 'champ.': 1,
 'changing': 1,
 'chanp': 1,
 'clean': 1,
 'comparison,': 1,
 'deffinite': 1,
 'diaper': 7,
 'difficult': 1,
 'dirty': 1,
 'easy': 2,
 'economical,': 1,
 'edge': 1,
 'effective,': 1,
 'eminating': 1,
 'fabulous.updatei': 1,
 'flat.': 1,
 'fluerville': 1,
 'for': 2,
 'found': 1,
 'free,': 1,
 "friend's": 1,
 'from': 1,
 'garbage': 1,
 'genie': 2,
 'genieplus': 1,
 'graco': 1,
 'handed': 1,
 'have': 1,
 'hesitated': 1,
 'house': 1,
 'i': 3,
 'if': 1,
 'in': 2,
 'integrated': 1,
 'into': 2,
 'is': 4,
 "isn't": 1,
 'knew': 1,
 'less': 1,
 'little': 1,
 'loved': 1,
 'luke': 1,
 'made.': 1,
 'needed': 1,
 'no': 1,
 'nursery.': 1,
 'odor': 1,
 'of': 2,
 'on': 1,
 'one': 3,
 'pack': 1,
 '